In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
inception_resnetv2 = \
    model_development.inception_resnetv2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = inception_resnetv2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.8279 - accuracy: 0.6427 - precision: 0.6523 - recall: 0.4576
Epoch 1: val_accuracy improved from -inf to 0.60896, saving model to ../models/weights\InceptionResNetv2_1.00.00_fs.hdf5
119/119 [==============================] - 40s 234ms/step - loss: 0.8279 - accuracy: 0.6427 - precision: 0.6523 - recall: 0.4576 - val_loss: 0.7289 - val_accuracy: 0.6090 - val_precision: 0.5842 - val_recall: 0.6810
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5667 - accuracy: 0.6844 - precision: 0.7159 - recall: 0.5064
Epoch 2: val_accuracy did not improve from 0.60896
119/119 [==============================] - 21s 176ms/step - loss: 0.5667 - accuracy: 0.6844 - precision: 0.7159 - recall: 0.5064 - val_loss: 0.7317 - val_accuracy: 0.5910 - val_precision: 0.5504 - val_recall: 0.8712
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5478 - accuracy: 0.7256 - precision: 0.7702 - recall: 0

In [6]:
##
#Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/InceptionResNetv2_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_sss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 49.5257 - accuracy: 0.5293 - precision: 0.4906 - recall: 0.9373
Epoch 1: val_accuracy improved from -inf to 0.53433, saving model to ../models/weights\InceptionResNetv2_1.00.00_sss.hdf5
119/119 [==============================] - 71s 466ms/step - loss: 49.5257 - accuracy: 0.5293 - precision: 0.4906 - recall: 0.9373 - val_loss: 23.1570 - val_accuracy: 0.5343 - val_precision: 0.5114 - val_recall: 0.9632
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 6.7381 - accuracy: 0.6575 - precision: 0.6262 - recall: 0.6109
Epoch 2: val_accuracy did not improve from 0.53433
119/119 [==============================] - 49s 412ms/step - loss: 6.7381 - accuracy: 0.6575 - precision: 0.6262 - recall: 0.6109 - val_loss: 26.5736 - val_accuracy: 0.5134 - val_precision: 0.5000 - val_recall: 0.9632
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 2.0269 - accuracy: 0.6871 - precision: 0.7500 - reca

In [7]:
##
# Clear memories after model training:
#
model_development.clear_memory()